## Calculation of transmutation rates expected for different elements. This is a rough estimate but can be used as a first indication.
### Step-by-step

1. Obtain the cross-sections plot of interest for the element in study, normally those that are bigger impact are (n,$\alpha$) and (n,p) for 14.1 MeV neutrons and (n, $\gamma$) for thermal neutrons.
2. Multiply these cross-section plots with the one with neutron flux at the wall to obtain other two plots that show the **number of reactions by those processes**. This is, one for (n,$\alpha$), one for (n,p) and one for (n, $\gamma$).
3. Knowing the full surface area of the first wall and the thickness of the expected layer calculate the **number of reactions** during a full power year (~9.125 days) and for 20fpy (the full expected duration of ITER operation).
4. Calculate the **percentual loss of material** during the full power year and 20fpy.

In [187]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import os
import matplotlib.pyplot as plt

In [188]:
#Read neutron flux at ITER wall
neutron_flux = pd.read_csv('neutron_energy_spectrum.csv', names=['Neutron energy (eV)', 'Neutron flux (n cm-2 s-1)'], 
dtype={'Neutron energy (eV)': float, 'Neutron flux (n cm-2 s-1)': float}).to_numpy()

neutron_flux_function = interp1d(neutron_flux[:,0], neutron_flux[:,1], kind='linear', fill_value="extrapolate")
new_X = 1e6
new_Y = interpolated_function(new_X)
formatted_Y = "{:.2e}".format(new_Y)
print(f"For X = {new_X}, interpolated Y = {formatted_Y}")

For X = 1000000.0, interpolated Y = 2.86e+13


### Elemental information

This should display the abudance if the element has isotopes and read the files of the cross section of the reactions with neutrons

In [189]:
Pt194_abundance=0.3286
Pt195_abundance=0.3378
Pt196_abundance=0.2521

Pt194_A = 193.96268 #g/mol
Pt195_A = 194.96477 #g/mol
Pt196_A = 195.96495 #g/mol

rho_Pt = 21.45 #g/cm^3
N_A = 6.022e23 #atoms/mol

avg_A = Pt194_abundance*Pt194_A + Pt195_abundance*Pt195_A + Pt196_abundance*Pt196_A
N_target = (rho_Pt * N_A)/avg_A

PtFolder = ['Pt194', 'Pt195', 'Pt196']
PtArrays = []

In [190]:
for folder in PtFolder:
    folder_data = []
    for filename in os.listdir(folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder, filename)
            # Use pandas to read the CSV file into a DataFrame
            df = pd.read_csv(file_path, skiprows=3, sep=';', names=['Incident energy (eV)', 'Cross-section (b)'], 
            dtype={'Incident energy (eV)': float, 'Cross-section (b)': float})

            # Convert the cross-section values from barns to cm²
            df['Cross-section (b)'] *= 1e-24
            
            # Convert the DataFrame to a numpy array
            data_array = df.to_numpy()
            folder_data.append(data_array)
    PtArrays.append(folder_data)

In [191]:
#First position is of each one of these arrays is (n,$\alpha$), second (n,$\gamma$) and third (n,p)
Pt194_CS = (PtArrays[0][0], PtArrays[0][1], PtArrays[0][2])
Pt195_CS = (PtArrays[1][0], PtArrays[1][1], PtArrays[1][2])
Pt196_CS = (PtArrays[2][0], PtArrays[2][1], PtArrays[2][2])

In [192]:
#Cross sections for each reaction
Pt194_alpha = Pt194_CS[0]
Pt194_gamma = Pt194_CS[1]
Pt194_p = Pt194_CS[2]

Pt195_alpha = Pt195_CS[0]
Pt195_gamma = Pt195_CS[1]
Pt195_p = Pt195_CS[2]

Pt196_alpha = Pt196_CS[0]
Pt196_gamma = Pt196_CS[1]
Pt196_p = Pt196_CS[2]


TENHO DE APAGAR ISTO E COLOCAR EM LISTAS! DEPOIS POSSO ITERAR REACAO A REACAO

### ITER parameters

In [193]:
first_wall_area = 6.5e6 #in cm^2
thickness1 = 1 #in cm
thickness2 = 1e-4 #in cm
fpy = 788400 #in seconds
fpy_full = fpy*20 #in seconds

### Transmutation reaction rate

To calculate the transmutation reaction rate one needs to know the number of nuclei per cm^3, the neutron flux and cross-section of the transmutation reaction, so that:

$R[cm^{-3} s^{-1}] = N[cm^{-3}] \times \sigma[cm^2] \times \Phi[cm^{-2} s^{-1}]$

With this, multiplying by the total volume of the first wall layer, whether that'd be a coating or bulk, we obtain the number of reactions per second. Then, multiplying by the fpy in seconds, one can calculate the number of transmutation reactions per full power year. Assuming a wall made of 100% of the material of study, we can compare the number of new transmutated atoms with the number of original atoms and compare how many of the original atoms transmutated.

In [194]:
Pt194_alpha[:,1]*neutron_flux_function(Pt194_alpha[:,0])*N_target*first_wall_area*thickness1*fpy

array([-2.24250034e+19, -2.17551487e+19, -2.11419190e+19, ...,
       -5.65729283e+24, -5.67430993e+24, -5.69045860e+24])